# **Library Import**

In [6]:
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')

  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [13 lines of output]
  C:\Users\ASUS\anaconda3\Lib\site-packages\setuptools\__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!
  
          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************
  
  !!
    dist.fetch_build_eggs(dist.setup_requires)
  error in pandas setup command: 'install_requires' must be a string or list of strings containing valid project/version requirement specifiers; Expected end or semicolon (after version specifier)
      pytz >= 2011k
           ~~~~~~~^
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a pro

ModuleNotFoundError: No module named 'google'

In [ ]:
import glob, random
import keras, os, sys
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import imageio
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from PIL import Image

from astropy.io import fits
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
from astropy.utils.data import get_pkg_data_filename



from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import accuracy_score

from sklearn.pipeline import make_pipeline


from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPool2D, Dense, Flatten, Layer, Input, Dropout
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = True
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras import initializers, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input

# **Data Import**

In [ ]:
exoTrain = glob.glob("/content/drive/MyDrive/exoplanet/Data/trainingData/ConfirmedExoplanets/*.png")
print ("Total of %d  images." % len(exoTrain))
print ('the filenames:\n')
print("\n".join(exoTrain[:5]))

In [ ]:
epTest = glob.glob("/content/drive/MyDrive/exoplanet/Data/testingData/ConfirmedExoplanets/*.png")
print ("Total of %d  images." % len(epTest))
print ('the filenames:\n')
print("\n".join(epTest[:5]))

In [ ]:
fpTrain = glob.glob("/content/drive/MyDrive/exoplanet/Data/trainingData/FalsePositiveExoplanets/*.png")
print ("Total of %d  images." % len(fpTrain))
print ('the filenames:\n')
print("\n".join(fpTrain[:5]))

In [ ]:
fpTest = glob.glob("/content/drive/MyDrive/exoplanet/Data/testingData/FalsePositiveExoplanets/*.png")
print ("Total of %d  images." % len(fpTest))
print ('the filenames:\n')
print("\n".join(fpTest[:5]))

In [ ]:
kepler = pd.read_csv('/content/drive/MyDrive/exoplanet/keplerData.csv', skiprows=144)
kepler.head()

In [ ]:
kepler = kepler.drop(['rowid'],1).reset_index(drop=True)
kepler.head()

# **Data Processing**

In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/exoplanet/Data/trainingData/'
TEST_DIR = '/content/drive/MyDrive/exoplanet/Data/testingData/'
CATEGORIES = ['ConfirmedExoplanets','FalsePositiveExoplanets']
IMG_SIZE=100
scores= []

In [ ]:
kepler.info()

In [ ]:
pd.DataFrame(round((kepler.isnull().sum() * 100/ len(kepler)),2).sort_values(ascending=False)).head(30)

In [ ]:
classes=kepler.koi_disposition.unique()
counts = kepler.koi_disposition.value_counts().to_list()
plt.bar(classes,counts)
print(kepler.koi_disposition.value_counts())

In [ ]:
# Dropping rows with more than 80% data missing
kepler = kepler.dropna(thresh=len(kepler) * .80, axis=1)
kepler.info()

In [ ]:
for column in kepler.columns:
    print(column)
    print(kepler[column].unique())

In [ ]:
kepler.drop(['kepid','kepoi_name','koi_vet_stat','koi_vet_date','koi_disp_prov','koi_pdisposition','koi_datalink_dvr',
                      'koi_parm_prov','koi_tce_delivname'], inplace=True, axis=1)

In [ ]:
kepler.dropna(inplace=True)
kepler.info()

# Label encoding

In [ ]:
le = LabelEncoder()

In [ ]:
kepler['koi_disposition'] = le.fit_transform(kepler['koi_disposition'])

In [ ]:
keplerX= kepler.drop(['koi_disposition'],axis=1)
keplerY = kepler['koi_disposition']

# Dimension Reduction

In [ ]:
pca = PCA(n_components=10)

In [ ]:
# fit and transform data
keplerX = pca.fit_transform(keplerX)
keplerX.shape

In [ ]:
sc = StandardScaler()
keplerX = sc.fit_transform(keplerX)
keplerX.shape

# Class Balancing

In [ ]:
oversample = SMOTE()
keplerX, keplerY = oversample.fit_resample(keplerX, keplerY)

In [ ]:
counter = Counter(keplerY)
for k,v in counter.items():
    per = v / len(keplerY) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
plt.bar(counter.keys(), counter.values())
plt.show()

# Astropy

In [ ]:
_, axs = plt.subplots(3, 5, figsize=(12, 12))
axs = axs.flatten()
for img, ax in zip(exoTrain, axs):
    image = Image.open(img)
    ax.imshow(image)
plt.show()

In [ ]:
_, axs = plt.subplots(3, 5, figsize=(12, 12))
axs = axs.flatten()
for img, ax in zip(fpTrain, axs):
    image = Image.open(img)
    ax.imshow(image)
plt.show()

In [ ]:
_, axs = plt.subplots(1, 4, figsize=(12, 12))
axs = axs.flatten()
for img, ax in zip(epTest, axs):
    image = Image.open(img)
    ax.imshow(image)
plt.show()

In [ ]:
_, axs = plt.subplots(1, 3, figsize=(12, 12))
axs = axs.flatten()
for img, ax in zip(fpTest, axs):
    image = Image.open(img)
    ax.imshow(image)
plt.show()

In [ ]:
confirmedExoFits = glob.glob("/content/drive/MyDrive/exoplanet/rawdata/confirmedExoplanets/010874614/*.fits")
print ("Total of %d  images." % len(confirmedExoFits))
print ('the filenames:\n')
print("\n".join(confirmedExoFits[:5]))

In [ ]:
fpExoFits = glob.glob("/content/drive/MyDrive/exoplanet/rawdata/falsePositiveExoplanets/000892772/*.fits")
print ("Total of %d  images." % len(fpExoFits))
print ('the filenames:\n')
print("\n".join(fpExoFits[:5]))

In [ ]:
for file in confirmedExoFits:
    hdul = fits.open(file)
    hdul.info()

In [ ]:
for file in fpExoFits:
    hdul = fits.open(file)
    hdul.info()

# **Data Augmentation**

In [ ]:
gen = ImageDataGenerator(rescale=1./255)
train = gen.flow_from_directory(directory=TRAIN_DIR, target_size=(224,224))

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()


augmented_images = [train[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
gen = ImageDataGenerator(rescale=1./255)
test = gen.flow_from_directory(directory=TEST_DIR, target_size=(224,224))

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()


augmented_images = [test[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
gen = ImageDataGenerator()
train = gen.flow_from_directory(directory=TRAIN_DIR, target_size=(224,224))
test = gen.flow_from_directory(directory=TEST_DIR, target_size=(224,224))

In [ ]:
modelScores=pd.DataFrame()

# **CNN**

In [ ]:
cnnmodel = Sequential()
cnnmodel.add(Conv2D(input_shape = (224, 224, 3), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
cnnmodel.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
cnnmodel.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))
cnnmodel.add(Flatten())
cnnmodel.add(Dense(units = 2, activation = "sigmoid"))
opt = Adam(learning_rate = 0.001)
cnnmodel.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy,   metrics = ['accuracy'])
cnnmodel.summary()

In [ ]:
history = cnnmodel.fit(train, validation_data= test, epochs=5)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = cnnmodel.evaluate(test, verbose=2)
print("Loss = ", loss)
accuracy= accuracy*100
print("Accuracy = ", accuracy)

In [ ]:
modelScores=modelScores.append([['CNN Image', accuracy]])
modelScores

# VGG16

In [ ]:
model = VGG16(include_top=False,  weights='imagenet',  input_shape=(224, 224, 3))
print(model.summary())

In [ ]:
vgg16 = model.output
vgg16 = Flatten()(vgg16)
vgg16 = Dense(4096, activation='relu')(vgg16)
vgg16 = Dense(1072, activation='relu')(vgg16)
vgg16 = Dropout(0.2)(vgg16)
output_layer = Dense(2, activation='softmax')(vgg16)
model = Model(inputs=model.input, outputs=output_layer)
opt = Adam(learning_rate = 0.001)

model.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy,   metrics = ['accuracy'])

In [ ]:
history = model.fit(train, validation_data= test, epochs=5)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(test, verbose=2)
print("Loss = ", loss)
accuracy= accuracy*100
print("Accuracy = ", accuracy)

In [ ]:
modelScores=modelScores.append([['VGG16 Image', accuracy]])
modelScores

# VGG19

In [ ]:
model = VGG19(include_top=False,  weights='imagenet',  input_shape=(224, 224, 3))
print(model.summary())

In [ ]:
vgg19 = model.output
vgg19 = Flatten()(vgg19)
vgg19 = Dense(1032, activation='relu')(vgg19)
vgg19 = Dropout(0.2)(vgg19)
output_layer = Dense(2, activation='softmax')(vgg19)
model = Model(inputs=model.input, outputs=output_layer)
opt = Adam(learning_rate = 0.001)

model.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy,   metrics = ['accuracy'])

In [ ]:
history = model.fit(train,steps_per_epoch=10, validation_data= test, validation_steps=5, epochs=5)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(test, verbose=2)
print("Loss = ", loss)
accuracy= accuracy*100
print("Accuracy = ", accuracy)

In [ ]:
modelScores=modelScores.append([['VGG19 Image', accuracy]])
modelScores

# Kepler Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(keplerX, keplerY, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# **CNN**

In [ ]:
xtrain = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
xtest = X_test.reshape((X_test.shape[0],X_test.shape[1],1))
y_train = np.array(y_train)
y_test = np.array(y_test)
xtrain.shape, xtest.shape, y_train.shape, y_test.shape

In [ ]:
cnnmodel = Sequential()
cnnmodel.add(Conv1D(input_shape = (xtrain.shape[1],1), filters = 256, kernel_size = 3, padding = "same", activation = "tanh"))
cnnmodel.add(Flatten())
cnnmodel.add(Dense(units = 256, activation = "tanh"))
cnnmodel.add(Dense(units = 128, activation = "tanh"))
cnnmodel.add(Dense(units = 4, activation = "sigmoid"))
cnnmodel.compile(optimizer = 'adam', loss= keras.losses.sparse_categorical_crossentropy, metrics = ['accuracy'])
cnnmodel.summary()

In [ ]:
history = cnnmodel.fit(xtrain, y_train, validation_data= (xtest, y_test), epochs=20)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = cnnmodel.evaluate(xtest, y_test, verbose=2)
print("Loss = ", loss)
accuracy= accuracy*100
print("Accuracy = ", accuracy)

In [ ]:
modelScores=modelScores.append([['CNN Kepler', accuracy]])
modelScores

# SVM

In [ ]:
svm = SVC(kernel='linear', C=0.5, gamma=6)

In [ ]:
svm.fit(X_train, y_train)

In [ ]:
pred = svm.predict(X_test)

In [ ]:
accuracy= np.round(accuracy_score(y_test, pred)*100,0)
print("Accuracy = ", accuracy)

In [ ]:
modelScores=modelScores.append([['SVM Kepler', accuracy]])
modelScores

# **Random Forest Classifier**

In [ ]:
rf = RandomForestClassifier(criterion='entropy', min_samples_split=100)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
pred = rf.predict(X_test)

In [ ]:
accuracy= np.round(accuracy_score(y_test, pred)*100,0)
print("Accuracy = ", accuracy)

In [ ]:
modelScores=modelScores.append([['Random Forest Trees Kepler', accuracy]])
modelScores

# **Stacking Classifier**

In [ ]:
estimators = [('rf', rf),  ('svc', make_pipeline(sc, svm))]
model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression() )

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_test)

In [ ]:
accuracy= np.round(accuracy_score(y_test, pred)*100,0)
print("Accuracy = ", accuracy)

In [ ]:
modelScores=modelScores.append([['Stacking Kepler', accuracy]])
modelScores

# Comparitive Analysis

In [ ]:
modelScores.columns = ['Model', 'Accuracy']
modelScores.sort_values(by= 'Accuracy', ascending=False)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(modelScores['Model'], modelScores['Accuracy'])
plt.bar(modelScores['Model'], modelScores['Accuracy'])
plt.xticks(rotation=55)
plt.xlabel('Models')
plt.ylabel('Accuracy Scores')